In [31]:
import pandas as pd
from xgboost import XGBRegressor
import warnings; warnings.filterwarnings('ignore')
from DatasetCreation import *
from tqdm import tqdm
import pickle as pkl

In [2]:
data,y = make_dataset('data_with_important_features.csv', True, False, True)

Using TensorFlow backend.


In [5]:
from collections import Counter

def topN(d,N):
    c = Counter()
    def cou(x):
        c[x]= c[x]+1
    d.map(cou)
    
    def setWord(x):
        if c[x] > N:
            return x
        else:
            return 'other'
    return d.map(setWord)

# Выбираем самых значимых

In [6]:
data['employer:name'] = topN(data['employer:name'] , 1000)

In [9]:
data['area:name'] = topN(data['area:name'], 1000)

In [11]:
data['name_0']=data['name'].iloc[0].split()[0]

# Векторизуем

In [12]:
#TODO: Учитывать Skill2Vec

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# категориальные колонки 
all_categorial = ['employer:name', 'name_0','specializations:0:profarea_name','specializations:1:profarea_name','specializations:2:profarea_name','specializations:3:profarea_name','specializations:4:profarea_name','specializations:5:profarea_name', 'area:name','employment:name','experience:name','schedule:name','billing_type:name']


data_categ = data[all_categorial]

# {имя колонки : LabelEncoder..}
encNames = {}

# учим LabelEncoder'ы и сохраняем их в словарь, а затем применяем энкодеры к all_data
for name in all_categorial:
    enc = LabelEncoder()
    enc.fit(data_categ[name]) 
    encNames[name] = enc
    data_categ[name] = enc.transform(data_categ[name])

# заполняем null
data_categ.fillna(-1, inplace=True)

ohe = OneHotEncoder(categorical_features=data_categ.columns.isin(all_categorial),sparse=True)

data_categ = ohe.fit_transform(data_categ)


In [13]:
data_categ.shape

(161544, 226)

# Достанем данные о предсказании прошлых нейронок

In [17]:
train_pred = pkl.load(open('Ponyland/X_train_pred.pkl','rb'))
test_pred = pkl.load(open('Ponyland/X_test_pred.pkl','rb'))

In [28]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
#Зарпллаты в логарифмизированном виде распределены практически нормально
Y = np.log(y+1)

X_train, X_val, y_train,y_val, yy_train, yy_val = train_test_split(data_categ.toarray(), Y,y,random_state=42,test_size=0.1)

In [20]:
X_train = np.hstack((X_train,train_pred))
X_val = np.hstack((X_val, test_pred))

# Обучаем модель

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier
import xgboost as xgb


params = { 
    'seed': 0, 
    'colsample_bytree': 0.8, 
    'silent': 1, 
    'subsample': 0.7, 
    'learning_rate': 0.03, 
    'objective': 'reg:linear', 
    'max_depth': 15, 
    'min_child_weight': 10, 
    'booster': 'gbtree'
} 

#search_X1, search_X2, search_y1, search_y2 = train_test_split(X_train, y_train, test_size=0.2)

d_train = xgb.DMatrix(X_train, label=y_train) 

#watchlist = [(d_train, 'train'), (d_valid, 'eval')] 



clf = xgb.train(params, 
d_train, 
250)


#reg = XGBClassifier(nthread=15,n_estimators = 200, max_depth=20,mea,learning_rate=0.05)
#reg = RandomForestRegressor(n_estimators=50)
#reg.fit(X_train,yy_train)




In [24]:

d_test = xgb.DMatrix(X_val, label=y_val) 

In [30]:
predictions = np.exp(clf.predict(d_test))-1

print np.sqrt(mean_squared_error(predictions,np.exp(y_val)-1)), 'MSE, в рублях'
print (np.abs(predictions - np.exp(y_val)+1)/((np.exp(y_val)-1))).mean(),'Средняя относительная ошибка'

20773.2845639 MSE, в рублях
0.201785861744 Средняя относительная ошибка
